# Dataset: City Inspections Records

In [1]:
import pandas as pd
inspections = pd.read_csv(
    'data/Food_Establishment_Inspections.csv', 
    usecols = ['BusinessName', 'ISSDTTM', 'EXPDTTM', 'LICSTATUS', 'LICENSECAT',
       'DESCRIPT', 'RESULT', 'RESULTDTTM', 'Violation', 'ViolLevel',
       'ViolDesc', 'VIOLDTTM', 'ViolStatus', 'StatusDate', 'Comments', 'Address', 'City', 'State', 'Zip', 'Location'
    ],
    parse_dates = ['ISSDTTM', 'EXPDTTM', 'RESULTDTTM', 'VIOLDTTM', 'StatusDate'], 
    infer_datetime_format = True
)
inspections.columns = [
    'name',           # business name
    'lic_issue',      # date of business license
    'lic_expire',     # expiration date of business license
    'lic_status',     # license status (active or inactive)
    'lic_category',   # license category ("FS", "FT", "RS", or "MFW")
    'type',           # more verbal description of license category 
    'result',         # result of the inspection (12 different possibilities)
    'result_date',    # date of inspection
    'violation',      # code for violation type
    'level',          # violation severity (can be * or ** or ***)
    'description',    # verbal description of violation
    'violation_date', # generally (or always?) the same as result_date
    'status',         # simply pass or fail
    'status_date',    # generally a day or so later than result_date, but often not given
    'comments',       # free-form text comments
    'address',
    'city',
    'state',
    'zip',
    'location'        # (latitude, longitude)
]
inspections['result_date'] = inspections.result_date.apply(lambda x: x.date())
inspections['location'] = inspections.location.str.strip('()')
inspections[['latitude','longitude']] = inspections['location'].apply(lambda x: pd.Series(str(x).split(',')))
inspections['latitude'] = pd.to_numeric(inspections.latitude, errors='coerce')
inspections['longitude'] = pd.to_numeric(inspections.longitude, errors='coerce')
inspections[['address']] = (
    inspections.address.fillna(inspections.name) + ', ' + inspections.city.fillna('Boston') + ', ' 
    + inspections.state.fillna('MA') + (' 0'+inspections.zip.fillna(0).astype(int).astype(str)).replace(' 00','')
)
inspections = inspections.drop(['city', 'state', 'zip', 'location'], 1)

In [24]:
inspections = inspections[
    (inspections.result_date.astype(str) >= '2011-08-01') 
]

In [17]:
inspections.groupby(['result','status']).count()[['name']]

name
result     status       
HE_Closure Fail       33
           Pass        2
HE_FAILNOR Fail        4
           Pass        8
HE_Fail    Fail    98904
           Pass     1207
HE_FailExt Fail     7847
           Pass    13032
HE_Filed   Fail    11304
           Pass    17076
HE_Hearing Fail     5015
           Pass     4183
HE_Misc    Fail        4
           Pass        4
HE_NotReq  Fail       32
HE_OutBus  Fail      268
           Pass      108
HE_Pass    Fail        1
           Pass    63083
HE_TSOP    Fail     2381
           Pass      104

In [27]:
pd.options.display.max_rows=1000
inspections.groupby(['result_date','name','level','result','type']).count()[['status']].sort_index()

status
result_date name                                          level result     type                                 
2011-08-01  ARAMARK @ SULLIVAN & WORCESTER                *     HE_Filed   Eating & Drinking                   2
            ARIRANG HOUSE                                 *     HE_Pass    Eating & Drinking                   7
                                                          **    HE_Pass    Eating & Drinking                   1
            Alexander's Wine & Spirit                     *     HE_Fail    Retail Food                         6
            Au Bon Pain                                   *     HE_Fail    Eating & Drinking w/ Take Out       2
                                                          ***   HE_Fail    Eating & Drinking w/ Take Out       1
            Beijing Taste Restaurant                      *     HE_FailExt Eating & Drinking w/ Take Out       4
                                                          **    HE_FailExt Eating & Drinking w/ Take Out       1
                                                          ***   HE_FailExt Eating & Drinking w/ Take Out       1
            CHARLEY'S EATING & DRINKING SALOON            *     HE_Fail    Eating & Drinking                   3
                                                          **    HE_Fail    Eating & Drinking                   1
            CHINA HOUSE                                   *     HE_Pass    Eating & Drinking w/ Take Out       2
                                                          **    HE_Pass    Eating & Drinking w/ Take Out       1
            CRAZY DOUGH'S PIZZA COMPANY                   *     HE_Fail    Eating & Drinking                   8
                                                          **    HE_Fail    Eating & Drinking                   1
            Caribe Restaurant                             *     HE_Fail    Eating & Drinking w/ Take Out      11
                                                          **    HE_Fail    Eating & Drinking w/ Take Out       2
                                                          ***   HE_Fail    Eating & Drinking w/ Take Out       1
            Courtyard By Marriott                         *     HE_FailExt Eating & Drinking                  10
                                                          **    HE_FailExt Eating & Drinking                   1
            Cristelle's Restaurant                        *     HE_Fail    Eating & Drinking w/ Take Out       1
                                                          ***   HE_Fail    Eating & Drinking w/ Take Out       1
            Dunkin Donuts (209 North Harvard St.)         *     HE_Pass    Eating & Drinking w/ Take Out       1
                                                          ***   HE_Pass    Eating & Drinking w/ Take Out       1
            El Progresso Market                           *     HE_Filed   Retail Food                         2
            FLORENTINE CAFFE                              *     HE_Fail    Eating & Drinking                   1
                                                          ***   HE_Fail    Eating & Drinking                   2
            Gennaro 5 North Square                        *     HE_Fail    Eating & Drinking                   3
            MARRIOTT HOTEL-COPLEY PLACE                   *     HE_Fail    Eating & Drinking                   5
                                                          ***   HE_Fail    Eating & Drinking                   2
            MUL'S DINER                                   *     HE_Fail    Eating & Drinking                   5
                                                          **    HE_Fail    Eating & Drinking                   1
                                                          ***   HE_Fail    Eating & Drinking                   2
            McDonald's                                    *     HE_Pass    Eating & Drinking w/ Take Out       6
                                      

In [29]:
inspections[inspections.name == 'Au Bon Pain'].sort_values('result_date')

,name,lic_issue,lic_expire,lic_status,lic_category,type,result,result_date,violation,level,description,violation_date,status,status_date,comments,address,latitude,longitude
195730,Au Bon Pain,2011-12-28 11:22:06,2011-12-31 23:59:00,Inactive,FT,Eating & Drinking w/ Take Out,HE_Fail,2011-08-01,23-4-602.13,*,Non-Food Contact Surfaces Clean,2011-08-01 14:48:35,Fail,NaT,Exterior and Interior of microwave heavily soi...,"100 HUNTINGTON AV, BOSTON, MA 02116",NaN,NaN
221090,Au Bon Pain,2011-12-28 11:22:06,2011-12-31 23:59:00,Inactive,FT,Eating & Drinking w/ Take Out,HE_Fail,2011-08-01,03-3-501.16(A),***,Cold Holding,2011-08-01 14:48:35,Fail,NaT,Fridge running between 50-52F Pre-cooked har...,"100 HUNTINGTON AV, BOSTON, MA 02116",NaN,NaN
223060,Au Bon Pain,2011-12-28 11:22:06,2011-12-31 23:59:00,Inactive,FT,Eating & Drinking w/ Take Out,HE_Fail,2011-08-01,37-6-501.11-.12,*,Improper Maintenance of Walls/Ceilings,2011-08-01 14:48:35,Fail,NaT,Walls soiled clean to remove.,"100 HUNTINGTON AV, BOSTON, MA 02116",NaN,NaN
253594,Au Bon Pain,2011-12-28 11:22:06,2011-12-31 23:59:00,Inactive,FT,Eating & Drinking w/ Take Out,HE_Pass,2011-08-15,23-4-602.13,*,Non-Food Contact Surfaces Clean,2011-08-15 10:37:35,Pass,2011-08-15 15:37:56,Exterior and Interior of microwave heavily soi...,"100 HUNTINGTON AV, BOSTON, MA 02116",NaN,NaN
189531,Au Bon Pain,2011-12-28 11:22:06,2011-12-31 23:59:00,Inactive,FT,Eating & Drinking w/ Take Out,HE_Pass,2011-08-15,03-3-501.16(A),***,Cold Holding,2011-08-15 10:37:35,Pass,2011-08-15 15:37:56,Fridge running between 50-52F Pre-cooked har...,"100 HUNTINGTON AV, BOSTON, MA 02116",NaN,NaN
243145,Au Bon Pain,2011-12-28 11:22:06,2011-12-31 23:59:00,Inactive,FT,Eating & Drinking w/ Take Out,HE_Pass,2011-08-15,37-6-501.11-.12,*,Improper Maintenance of Walls/Ceilings,2011-08-15 10:37:35,Pass,2011-08-15 15:37:56,Walls soiled clean to remove.,"100 HUNTINGTON AV, BOSTON, MA 02116",NaN,NaN
148874,Au Bon Pain,2011-12-28 09:40:30,2016-12-31 23:59:00,Active,FT,Eating & Drinking w/ Take Out,HE_Pass,2011-08-23,NaN,NaN,NaN,NaT,NaN,NaT,NaN,"26 PARK PLAZA, BOSTON, MA 02110",NaN,NaN
146072,Au Bon Pain,2011-12-28 11:11:41,2016-12-31 23:59:00,Active,FT,Eating & Drinking w/ Take Out,HE_Filed,2011-09-02,29-5-201/02.11,*,Installed and Maintained,2011-09-02 12:43:37,Fail,NaT,repair hot water knob to hand sink in main kit...,"101 Merrimac ST, Boston, MA 02114",42.363840,-71.061660
189535,Au Bon Pain,2011-09-13 11:34:08,2011-12-31 23:59:00,Inactive,FS,Eating & Drinking,HE_OutBus,2011-09-08,NaN,NaN,NaN,NaT,NaN,NaT,NaN,"800 Washington St, Boston, MA 02111",42.349591,-71.063586
148494,Au Bon Pain,2011-12-28 09:54:27,2016-12-31 23:59:00,Active,FS,Eating & Drinking,HE_Fail,2011-09-30,03-3-501.16(A),***,Cold Holding,2011-09-30 11:58:56,Fail,NaT,muesli 48 degrees/provide proper temperature o...,"360 HUNTINGTON AV, BOSTON, MA 02115",NaN,NaN


In [1]:
import pandas as pd
from numpy import nan
import numpy as np
import scipy.stats as stats
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
%matplotlib inline

# Datasets from Yelp contest

In [2]:
import json
from pandas.io.json import json_normalize

with open('data/yelp_academic_dataset_business.json', 'r') as f:
    data = f.readlines()
data = map(lambda x: x.rstrip(), data)
data_json_str = '[' + (',').join(data) + ']'

business = pd.read_json(data_json_str)
business = business[[
    'name',
    'full_address',
    'city',
    'open',
    'business_id',
    'latitude',
    'longitude'
]]
business.columns = [
    'name',     
    'address',
    'city',
    'open',
    'yelp_id',
    'latitude',
    'longitude'
]
business[['address']] = business.address.str.replace('\n','GZRP').str.replace(r'GZRP.*GZRP', ', ').str.replace('GZRP', ', ')

%reset_selective -f data_json_str

In [3]:
idtable = pd.read_csv(
    'data/restaurant_ids_to_yelp_ids.csv'
)
idtable = pd.concat([
        idtable[['restaurant_id','yelp_id_0']].rename(columns = {'yelp_id_0':'yelp_id'}),
        idtable[idtable.yelp_id_1.notnull()][['restaurant_id','yelp_id_1']].rename(columns = {'yelp_id_1':'yelp_id'}),
        idtable[idtable.yelp_id_2.notnull()][['restaurant_id','yelp_id_2']].rename(columns = {'yelp_id_2':'yelp_id'}),
        idtable[idtable.yelp_id_3.notnull()][['restaurant_id','yelp_id_3']].rename(columns = {'yelp_id_3':'yelp_id'})
], ignore_index = True)

In [4]:
violations = pd.read_csv(
    'data/AllViolations.csv', 
    parse_dates = ['date'], 
    infer_datetime_format = True,
    index_col = 0
)

In [5]:
violations = violations.merge(
    idtable, 
    on = 'restaurant_id'
).merge(
    business, 
    on = 'yelp_id'
).drop(['yelp_id', 'name', 'address', 'city', 'open'], 1)

In [6]:
violations = violations[violations.date >= '2011-08-01'].sort_values('date')

In [7]:
violations

,date,restaurant_id,*,**,***,latitude,longitude
15836,2011-08-01,NbE1Bk3J,8,1,0,42.346810,-71.088960
1401,2011-08-01,7RO5vjEq,7,1,0,42.346246,-71.087049
25218,2011-08-01,V430D43B,1,0,2,42.364263,-71.053833
34217,2011-08-01,Y1EmaVEw,7,0,1,42.351813,-71.062679
27586,2011-08-01,6Wo2Nyo9,11,2,1,42.315208,-71.066018
33235,2011-08-01,8x3zgYok,2,0,1,42.349664,-71.070235
6368,2011-08-01,8x3zx2Ok,5,0,2,42.347022,-71.079289
21033,2011-08-01,qN3gvnEA,5,0,1,42.351503,-71.060239
21523,2011-08-01,VpoG57Er,1,0,0,42.255668,-71.123935
6157,2011-08-01,dj3dlN39,1,0,2,42.355682,-71.060278


# Dataset: 311 service calls

In [8]:
services = pd.read_csv(
    'data/311__Service_Requests.csv', 
    usecols = [
       'OPEN_DT', 'CLOSED_DT', 'CASE_STATUS', 'CLOSURE_REASON', 'CASE_TITLE', 'SUBJECT', 'REASON',
       'TYPE', 'Department', 'LATITUDE', 'LONGITUDE'
    ],
    parse_dates=['OPEN_DT', 'CLOSED_DT'], 
    infer_datetime_format = True
)
services.columns = [
    'open',         # date the complaint was registered
    'closed',       # date the complaint was resolved
    'status',       # open or closed
    'closure',      # details of how the complaint was closed (NaN if open)
    'title',        # description of the issue
    'subject',      # category of the issue (e.g. public works, civil rights, animal control, etc.)
    'reason',       # more specific category (often very similar to the "title")
    'type',         # usually identical to the "title" 
    'department',   # usually an abbreviation of the "subject"
    'latitude',      
    'longitude'      
]

In [9]:
services.open.min()

Timestamp('2011-07-01 01:32:33')

In [10]:
services = services.sort_values(['open', 'reason'])

In [11]:
reason = services.reason.value_counts().head(45).reset_index()['index']

In [12]:
for j in range(45):
    violations[reason[j]] = nan

violations['delay'] = nan
    
for row in range(len(violations)):
    c = (violations.latitude.iloc[row], violations.longitude.iloc[row])
    ri = violations.restaurant_id.iloc[row]
    t = violations.date.iloc[row]
    tlast = violations[(violations.date < t - timedelta(days = .5)) & (violations.restaurant_id == ri)].date.max()
    violations['delay'].iloc[row] = t - tlast
    s = services[
            (services.open < t - timedelta(days = 5)) &
            (services.open > t - timedelta(days = 15)) &
            ((services.latitude - c[0])**2 + (services.longitude - c[1])**2 < .0025)
    ]
    for j in range(45):
        violations.iloc[row,j+7] = sum(
            s.reason == reason[j]
        )
        
violations['delay'] = (violations.delay) / (timedelta(days=1))
violations.delay = violations.delay.astype(float)

C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


## distance < .02, 0 to 28 days back:

In [48]:
violations[['*','**','***'] + list(reason[0:20])].corr().iloc[0:3,3:23]

,Sanitation,Highway Maintenance,Street Cleaning,Street Lights,Signs & Signals,Housing,Recycling,Enforcement & Abandoned Vehicles,Building,Trees,Graffiti,Employee & General Comments,Environmental Services,Park Maintenance & Safety,Code Enforcement,Administrative & General Requests,Animal Issues,Notification,Health,Call Inquiry
*,0.091985,0.098900,0.034889,-0.055450,0.067711,0.065982,0.099547,0.111843,0.072037,0.087952,0.063599,0.034554,0.046607,0.069147,0.022731,0.058099,0.099414,0.013732,0.054429,0.016100
**,0.049136,0.031935,0.004057,-0.047720,0.036333,0.053365,0.031850,0.045815,0.026910,0.078773,0.052375,-0.013158,0.042866,0.013573,0.011590,0.016668,0.043813,-0.002690,0.015156,-0.009603
***,0.035112,0.059881,0.010786,-0.056769,0.047430,0.021063,0.018500,0.067096,0.041311,0.037885,0.066462,0.034638,0.043593,0.090469,0.013248,0.029524,0.090735,-0.014530,0.029344,0.033881


## distance < .01, 10 to 30 days back:

In [11]:
violations[['*','**','***'] + list(reason[0:20])].corr().iloc[0:3,3:23]

,Sanitation,Highway Maintenance,Street Cleaning,Street Lights,Signs & Signals,Housing,Recycling,Enforcement & Abandoned Vehicles,Building,Trees,Graffiti,Employee & General Comments,Environmental Services,Park Maintenance & Safety,Code Enforcement,Administrative & General Requests,Animal Issues,Notification,Health,Call Inquiry
*,0.091627,0.082441,0.038310,-0.044980,0.053826,0.076147,0.063700,0.067072,0.061207,0.083791,0.050795,0.063073,0.048805,0.076770,0.005621,0.058580,0.071621,0.055252,0.060314,0.009103
**,0.051607,0.023896,0.002531,-0.049226,0.023101,0.059979,0.022660,0.012486,0.022593,0.064981,0.046907,0.002647,0.034972,0.020570,0.006467,0.012397,0.029492,0.021803,0.021136,-0.014648
***,0.039754,0.062163,0.012167,-0.039329,0.056727,0.038150,0.009355,0.043080,0.027935,0.036528,0.059681,0.068344,0.057378,0.107348,0.008280,0.030810,0.070632,0.043646,0.061331,0.022034


## distance < .01, 5 to 20 days back:

In [32]:
violations[['*','**','***'] + list(reason[0:20])].corr().iloc[0:3,3:23]

,Sanitation,Highway Maintenance,Street Cleaning,Street Lights,Signs & Signals,Housing,Recycling,Enforcement & Abandoned Vehicles,Building,Trees,Graffiti,Employee & General Comments,Environmental Services,Park Maintenance & Safety,Code Enforcement,Administrative & General Requests,Animal Issues,Notification,Health,Call Inquiry
*,0.096086,0.082695,0.013918,-0.018246,0.059709,0.089361,0.090708,0.046378,0.064506,0.087838,0.031765,0.052285,0.061368,0.066074,0.003514,0.042632,0.071710,0.061767,0.069242,0.012038
**,0.055874,0.022744,-0.003431,-0.027590,0.029358,0.075034,0.035967,0.005668,0.024225,0.064767,0.031231,-0.001965,0.040540,0.018196,0.007398,0.017262,0.032847,0.031471,0.023901,-0.014900
***,0.043988,0.062389,-0.007273,-0.015515,0.061338,0.044851,0.029831,0.029591,0.034501,0.040312,0.052534,0.054484,0.072018,0.096915,0.004186,0.019291,0.070733,0.048527,0.065915,0.022316


## distance < .02, 5 to 15 days back:

In [37]:
violations[['*','**','***'] + list(reason[0:20])].corr().iloc[0:3,3:23]

,Sanitation,Highway Maintenance,Street Cleaning,Street Lights,Signs & Signals,Housing,Recycling,Enforcement & Abandoned Vehicles,Building,Trees,Graffiti,Employee & General Comments,Environmental Services,Park Maintenance & Safety,Code Enforcement,Administrative & General Requests,Animal Issues,Notification,Health,Call Inquiry
*,0.106908,0.115986,0.001879,0.006157,0.074328,0.082785,0.107119,0.089905,0.077322,0.081871,0.038815,0.023104,0.076547,0.044586,0.010754,0.040990,0.088298,0.030865,0.084600,0.016766
**,0.060457,0.041569,-0.011675,-0.003530,0.039156,0.065181,0.044649,0.036684,0.030010,0.062860,0.034747,-0.017938,0.061448,0.004176,0.002773,0.016096,0.040838,0.011197,0.039499,-0.010969
***,0.050463,0.070998,-0.016632,-0.005721,0.056877,0.038902,0.037944,0.053113,0.049424,0.042786,0.052627,0.021212,0.065170,0.069595,-0.001044,0.019070,0.083459,-0.002153,0.053125,0.032601


## distance < .05, 5 to 15 days back:

In [40]:
violations[['*','**','***'] + list(reason[0:20])].corr().iloc[0:3,3:23]

,Sanitation,Highway Maintenance,Street Cleaning,Street Lights,Signs & Signals,Housing,Recycling,Enforcement & Abandoned Vehicles,Building,Trees,Graffiti,Employee & General Comments,Environmental Services,Park Maintenance & Safety,Code Enforcement,Administrative & General Requests,Animal Issues,Notification,Health,Call Inquiry
*,0.108451,0.154236,-0.010561,-0.009663,0.089541,0.074555,0.131223,0.105753,0.112984,0.105726,0.094344,0.041794,0.118458,0.083360,-0.002358,0.061344,0.096560,0.008102,0.128420,0.050346
**,0.075268,0.075548,-0.010698,-0.000860,0.054662,0.049962,0.080476,0.058439,0.062179,0.074257,0.065940,0.015868,0.108027,0.035418,0.002446,0.034768,0.038519,0.013470,0.088006,0.010504
***,0.047084,0.073965,-0.021544,-0.038988,0.046897,0.039782,0.063104,0.046075,0.062928,0.059040,0.075355,0.004038,0.076891,0.056652,-0.003143,0.027932,0.079118,-0.013438,0.071253,0.029721


## distance < .05, 5 to 15 days back, excluding repeats:

In [185]:
violations[violations.delay > 45][['*','**','***'] + list(reason[0:45]) + ['delay']].corr().iloc[0:3,3:49]

,Sanitation,Highway Maintenance,Street Cleaning,Street Lights,Signs & Signals,Housing,Recycling,Enforcement & Abandoned Vehicles,Building,Trees,...,Water Issues,Alert Boston,Volunteer & Corporate Groups,Generic Noise Disturbance,Pothole,Boston Bikes,Parking Complaints,Cemetery,Office of The Parking Clerk,delay
*,0.139246,0.195804,-0.007455,-0.003573,0.119235,0.099797,0.157217,0.135949,0.157053,0.177941,...,0.114493,-0.028018,0.105806,0.033354,0.117984,0.036522,0.042902,0.083069,0.059529,0.016038
**,0.092148,0.110467,0.004545,-0.007807,0.084856,0.067311,0.098524,0.076487,0.093464,0.112405,...,0.065580,-0.018059,0.078871,0.001561,0.058061,0.018840,0.025826,0.066199,0.027245,0.012783
***,0.083102,0.125820,-0.020036,-0.029770,0.081129,0.066252,0.087781,0.081670,0.100093,0.117873,...,0.067391,-0.035587,0.062146,0.012172,0.091903,0.023659,0.019698,0.042357,0.023871,0.014625


In [24]:
stats.pearsonr(
    violations['***'],
    violations[reason[36]]
)

(0.037242371648445592, 7.9708185840462616e-07)

In [18]:
reason

0                            Sanitation
1                   Highway Maintenance
2                       Street Cleaning
3                         Street Lights
4                       Signs & Signals
5                               Housing
6                             Recycling
7      Enforcement & Abandoned Vehicles
8                              Building
9                                 Trees
10                             Graffiti
11          Employee & General Comments
12               Environmental Services
13            Park Maintenance & Safety
14                     Code Enforcement
15    Administrative & General Requests
16                        Animal Issues
17                         Notification
18                               Health
19                         Call Inquiry
20                           Operations
21     Traffic Management & Engineering
22                               Survey
23                           Disability
24                           Catchbasin


In [30]:
len(
    violations[(violations.delay > 45) & (violations['*'] + violations['**'] + violations['***'] == 0)]
) / len(violations[violations.delay > 45])

0.36364725617062066